In [245]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/My Drive/Colab Notebooks'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks



Conor Hoekstra

https://github.com/codereport/Talks/tree/master/2020-04-PyCon/BeautifulPythonRefactoring


In [246]:
import requests
import lxml.html as lh
import pandas as pd

In [247]:
url = 'http://pokemondb.net/pokedex/all'

In [248]:
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

[len(T) for T in tr_elements[:10]]

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [249]:
col = []
i = 0
for t in tr_elements[0]:
  i += 1
  name = t.text_content()
  print('%d:"%s"'%(i, name))
  col.append((name, []))

1:"#"
2:"Name"
3:"Type"
4:"Total"
5:"HP"
6:"Attack"
7:"Defense"
8:"Sp. Atk"
9:"Sp. Def"
10:"Speed"


In [250]:
for j in range(1, len(tr_elements)):
  T = tr_elements[j]
  if len(T) != 10:
    break
  i = 0
  for t in T.iterchildren():
    data = t.text_content()
    if i > 0:
      try:
        data = int(data)
      except:
        pass
    col[i][1].append(data)
    i+=1

[len(C) for (title,C) in col]

[1034, 1034, 1034, 1034, 1034, 1034, 1034, 1034, 1034, 1034]

In [251]:
Dict = {title:column for (title,column) in col}

In [252]:
df = pd.DataFrame(Dict)
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,001,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,002,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,003,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,003,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,004,Charmander,Fire,309,39,52,43,60,50,65


### Now let's refactor

In [253]:
# enumerate
col = []
for i,t in enumerate(tr_elements[0]):
  name = t.text_content()
  print('%d:"%s"'%(i, name))
  col.append((name, []))

0:"#"
1:"Name"
2:"Type"
3:"Total"
4:"HP"
5:"Attack"
6:"Defense"
7:"Sp. Atk"
8:"Sp. Def"
9:"Speed"


In [254]:
# delete print and enumerate
col = []
for t in tr_elements[0]:
  name = t.text_content()
  col.append((name, []))

col

[('#', []),
 ('Name', []),
 ('Type', []),
 ('Total', []),
 ('HP', []),
 ('Attack', []),
 ('Defense', []),
 ('Sp. Atk', []),
 ('Sp. Def', []),
 ('Speed', [])]

In [255]:
col = []
for t in tr_elements[0]: col.append((t.text_content(), []))

col

[('#', []),
 ('Name', []),
 ('Type', []),
 ('Total', []),
 ('HP', []),
 ('Attack', []),
 ('Defense', []),
 ('Sp. Atk', []),
 ('Sp. Def', []),
 ('Speed', [])]

In [256]:
# list comprehension
col = []
col = [(t.text_content(), []) for t in tr_elements[0]]

col

[('#', []),
 ('Name', []),
 ('Type', []),
 ('Total', []),
 ('HP', []),
 ('Attack', []),
 ('Defense', []),
 ('Sp. Atk', []),
 ('Sp. Def', []),
 ('Speed', [])]

In [257]:
# A. delete if
for j in range(1, len(tr_elements)):
  T = tr_elements[j]
  i = 0
  for t in T.iterchildren():
    data = t.text_content()
    if i > 0:
      try:
        data = int(data)
      except:
        pass
    col[i][1].append(data)
    i+=1

[len(C) for (title,C) in col]

[1034, 1034, 1034, 1034, 1034, 1034, 1034, 1034, 1034, 1034]

In [258]:
# B. use slicing (not j)
for T in tr_elements[1:]:
  i = 0
  for t in T.iterchildren():
    data = t.text_content()
    if i > 0:
      try:
        data = int(data)
      except:
        pass
    col[i][1].append(data)
    i+=1

[len(C) for (title,C) in col]

[2068, 2068, 2068, 2068, 2068, 2068, 2068, 2068, 2068, 2068]

In [259]:
# C: delete if
for T in tr_elements[1:]:
  for i,t in enumerate(T.iterchildren()):
    data = t.text_content()
    try:
      data = int(data)
    except:
      pass
    col[i][1].append(data)

[len(C) for (title,C) in col]

[3102, 3102, 3102, 3102, 3102, 3102, 3102, 3102, 3102, 3102]

In [260]:
# D: use conditional expresssion
for T in tr_elements[1:]:
  for i,t in enumerate(T.iterchildren()):
    data = t.text_content()
    data = int(data) if data.isnumeric() else data
    col[i][1].append(data)

[len(C) for (title,C) in col]

[4136, 4136, 4136, 4136, 4136, 4136, 4136, 4136, 4136, 4136]

In [261]:
# E: redundant line
for T in tr_elements[1:]:
  for i,t in enumerate(T.iterchildren()):
    data = t.text_content()
    col[i][1].append(int(data) if data.isnumeric() else data)

[len(C) for (title,C) in col]

[5170, 5170, 5170, 5170, 5170, 5170, 5170, 5170, 5170, 5170]

In [262]:
# put it all together
url = 'http://pokemondb.net/pokedex/all'

page = requests.get(url)                        # page handle
doc  = lh.fromstring(page.content)              # website contents
tr   = doc.xpath('//tr')                        # html <tr> table
col  = [(t.text_content(), []) for t in tr[0]]  # column titles

# scrape data
for T in tr[1:]:
  for i,t in enumerate(T.iterchildren()):
    data = t.text_content()
    col[i][1].append(int(data) if data.isnumeric() else data)

Dict = {title:column for (title,column) in col}
df = pd.DataFrame(Dict)

df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,3,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,309,39,52,43,60,50,65


In [263]:
url = 'http://pokemondb.net/pokedex/all'

page    = requests.get(url)                        # page handle
doc     = lh.fromstring(page.content)              # website contents
tr      = doc.xpath('//tr')                        # html <tr> table
titles  = [t.text_content() for t in tr[0]]        # column titles

# scrape data
fmt = lambda data : int(data) if data.isnumeric() else data
cols = zip(*[[fmt(t.text_content()) for t in T.iterchildren()] for T in tr[1:]])

Dict = {title:column for title,column in zip(titles,cols)}
df = pd.DataFrame(Dict)
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,3,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,309,39,52,43,60,50,65


In [268]:
header = {"User-Agent": "Mozilla /5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest" }
r  = requests.get(url, headers=header)
df = pd.read_html(r.text)[0]

df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,3,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,309,39,52,43,60,50,65


In [269]:
df = pd.read_html(requests.get(url).text)[0]
df.head()

,#,Name,Type,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
0,1,Bulbasaur,Grass Poison,318,45,49,49,65,65,45
1,2,Ivysaur,Grass Poison,405,60,62,63,80,80,60
2,3,Venusaur,Grass Poison,525,80,82,83,100,100,80
3,3,Venusaur Mega Venusaur,Grass Poison,625,80,100,123,122,120,80
4,4,Charmander,Fire,309,39,52,43,60,50,65
